In [58]:
import torch as th
import numpy as np
import pandas as pd

import os
import sys

from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score

from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import gc
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
EPOCHS = 1000
BATCH_SIZE = 2048
EARLY_STOP_ROUNDS = 30
USE_GPU = True

In [47]:
class UtilityLoss(nn.Module):
    def forward(self, day, weight, resp, action):
        cnt = day.max() - day.min() + 1
        nday = th.tensor(len(day.unique()), device=day.device, dtype=th.float32)
        C = th.sqrt(250. / nday)
        C6 = th.tensor(6., device=day.device)
        C0 = th.tensor(0., device=day.device)
        
        p = (weight * resp * action).reshape(-1, 1)
        m = th.zeros((int(cnt), len(p)), device=day.device)
        m[(day-day.min()).long(), th.arange(len(p))] = 1
        pi = th.mm(m, p)
        t = pi.sum() / (pi**2).sum().sqrt() * C
        u = th.clamp(t, C0, C6) * pi.sum() 
        return -u
def utility_score(day, weight, resp, action):
    day = day.reshape(-1, 1)
    p = (weight*resp*action).reshape(-1, 1)
    df = pd.DataFrame(np.hstack([day, p]), columns=['date', 'p'])
    
    p_i = df.groupby('date').sum().values
    t = (p_i.sum() / np.sqrt((p_i**2).sum())) * (np.sqrt(250 / len(p_i)))
    return np.clip(t, 0, 6) * p_i.sum()

In [72]:
class MLPModel(nn.Module):
    def __init__(self, nemb=16, nfea=129, act='LeakyReLU', output_act='softsign', **kwargs):
        super().__init__()
        self.emb = nn.Embedding(3, nemb)
        if act == 'LeakyReLU':
            act = nn.LeakyReLU
        elif act == 'Swish':
            act = lambda x: x * th.sigmoid(x)
        self.m = nn.Sequential(nn.Linear(nfea+nemb, 384), nn.BatchNorm1d(384), nn.LeakyReLU(), nn.Dropout(0.1),
                               nn.Linear(384, 896), nn.BatchNorm1d(896), nn.LeakyReLU(), nn.Dropout(0.2),
                               nn.Linear(896, 896), nn.BatchNorm1d(896), nn.LeakyReLU(), nn.Dropout(0.25),
                               nn.Linear(896, 384), nn.BatchNorm1d(384), nn.LeakyReLU(), nn.Dropout(0.22),
                               nn.Linear(384, 1))
        self.output_act = output_act
        
    def forward(self, x):
        o1 = self.emb(x[:, 0].long())
        x = th.cat([o1, x[:, 1:]], dim=1)
        o = self.m(x).reshape(-1)
        if self.output_act == 'softsign':
            return (nn.functional.softsign(o)+1)/2
        elif self.output_act == 'sigmoid':
            return th.sigmoid(o)
        else:
            raise NotImplementedError()
    
    def save(self, fname):
        th.save(self.state_dict(), fname)
        
    def load(self, fname, map_loc):
        self.load_state_dict(th.load(fname, map_location=map_loc))
    


In [44]:
def train_one_epoch(model, x, opt, batch_size):
    model.train()
    train_dl = DataLoader(x, batch_size=batch_size)
    #loss_fn = UtilityLoss()
    loss_fn = nn.BCELoss()
    pbar = tqdm(enumerate(train_dl))
    mean_acc = 0.
    mean_auc = 0.
    mean_score = 0.
    for i, (batx, dwra) in pbar:
        opt.zero_grad()
        pred = model(batx)
        #loss = loss_fn(dwra[:, 0], dwra[:, 1], dwra[:, 2], pred)
        loss = loss_fn(pred, dwra[:, 3])
        loss.backward()
        opt.step()
        y_true, y_pred = dwra[:, 3].cpu().numpy(), pred.cpu().detach().numpy()>0.5
        mean_acc = (accuracy_score(y_true, y_pred) + i*mean_acc)/(1+i)
        mean_auc = (roc_auc_score(y_true, y_pred) + i*mean_auc)/(1+i)
        mean_score = (utility_score(dwra[:, 0].cpu().numpy(), dwra[:, 1].cpu().numpy(), dwra[:, 2].cpu().numpy(), y_pred) + i*mean_score)/(1+i)
        pbar.set_description(f'train loss: {float(loss)}, acc: {mean_acc}, auc: {mean_auc}, uscore: {mean_score}')
    return float(loss), mean_acc, mean_auc, mean_score
def validate(model, x, dwra):
    model.eval()
    #loss_fn = UtilityLoss()
    loss_fn = nn.BCELoss()
    with th.no_grad():
        pred = model(x)
        #loss = loss_fn(dwra[:, 0], dwra[:, 1], dwra[:, 2], pred)
        loss = loss_fn(pred, dwra[:, 3])
        y_true, y_pred = dwra[:, 3].cpu().numpy(), pred.cpu().detach().numpy()>0.5
        auc = roc_auc_score(y_true, y_pred)
        acc = accuracy_score(y_true, y_pred)
        score = utility_score(dwra[:, 0].cpu().numpy(), dwra[:, 1].cpu().numpy(), dwra[:, 2].cpu().numpy(), y_pred)
        print(f'val loss: {float(loss)}, acc: {acc}, auc: {auc}, uscore: {score}')
        return float(loss), acc, auc, score

In [23]:
data = pd.read_csv('./input/train.csv')

features = [c for c in data.columns if 'feature' in c]

print('Filling...')
f_mean = data[features[1:]].mean()
data['feature_0'] = (data['feature_0'] + 1) // 2
data = data.query('weight > 0').reset_index(drop = True)
data[features[1:]] = data[features[1:]].fillna(f_mean)
data['action'] = (data['resp'] > 0).astype('int')

In [73]:
def process(data, model_params={}):

    train, val = train_test_split(data, train_size=0.7, shuffle=False)
    X_train, X_val = train[features].values, val[features].values
    y_train, y_val = train['action'].values, val['action'].values
    
    model = MLPModel(output_act='sigmoid', **model_params)
    print(model)
    opt = th.optim.Adam(model.parameters())
    if USE_GPU:
        model = model.cuda()
#         opt = opt.cuda()
        dev = 'cuda'
    else:
        dev = 'cpu'
    train_ds = TensorDataset(th.tensor(X_train.astype('float32'), device=dev), th.tensor(train[['date', 'weight', 'resp', 'action']].values.astype('float32'), device=dev))
    val_ds = th.tensor(X_val.astype('float32'), device=dev), th.tensor(val[['date', 'weight', 'resp', 'action']].values.astype('float32'), device=dev)
    opt_score, opt_epoch = 1e9, 0
    for e in range(EPOCHS):
        train_one_epoch(model, train_ds, opt, BATCH_SIZE)
        val_metrics = validate(model, *val_ds)
        
        if val_metrics[2] > opt_score:
            opt_score = val_metrics[2]
            opt_epoch = e
            model.save(f'mlp.pt')
        else:
            if e - opt_epoch >= EARLY_STOP_ROUNDS:
                break
    print(opt_score)
    return opt_score

In [74]:
process(data, {'act': 'Swish'})

train loss: 0.6916323304176331, acc: 0.5116306386152294, auc: 0.5096464768407564, uscore: 25.87422348182517: : 678it [00:54, 12.56it/s] 
train loss: 0.6917372345924377, acc: 0.550048828125, auc: 0.540932922903959, uscore: 55.719857946038246: : 2it [00:00, 14.53it/s]

val loss: 0.6932085156440735, acc: 0.5078492631904803, auc: 0.5098167218105776, uscore: 98.46987642834199


train loss: 0.6899170875549316, acc: 0.5183415216953804, auc: 0.5152739000921064, uscore: 29.830547379635842: : 678it [00:52, 12.94it/s]
train loss: 0.6894217729568481, acc: 0.5732421875, auc: 0.5672125908722461, uscore: 62.6304566860199: : 2it [00:00, 14.76it/s]

val loss: 0.6923532485961914, acc: 0.5166364674866711, auc: 0.5172247544855237, uscore: 1065.5180020129555


train loss: 0.6893985271453857, acc: 0.520755682367877, auc: 0.5178799885037118, uscore: 30.86498887595029: : 678it [00:52, 12.86it/s]  
train loss: 0.6882034540176392, acc: 0.579833984375, auc: 0.5752535289768342, uscore: 63.95200824737549: : 2it [00:00, 11.77it/s]

...

train loss: 0.6779178380966187, acc: 0.5568458830984041, auc: 0.5550816268926958, uscore: 44.87796386323589: : 678it [00:52, 12.91it/s] 
train loss: 0.6837069988250732, acc: 0.552734375, auc: 0.5479156496945194, uscore: 52.5649938583374: : 2it [00:00, 14.56it/s]

val loss: 0.6974143385887146, acc: 0.5121309853681187, auc: 0.5129385932464595, uscore: 621.7768309260459


train loss: 0.6764249801635742, acc: 0.5590207732661403, auc: 0.5572051076496235, uscore: 46.13847756493668: : 678it [00:53, 12.79it/s] 


val loss: 0.69776850938797, acc: 0.5120788307954245, auc: 0.5129928410264264, uscore: 719.1014589587371
0.5098167218105776


0.5098167218105776

In [62]:
th.optim.Adam?